# Propensity Score Estimation: P(Email | Customer Features)

## 🎯 Learning Objectives

In this notebook, we will:
1. **Understand propensity scores** - What they are and why they matter
2. **Estimate propensity scores** - Build a logistic regression model
3. **Diagnose the model** - Check common support and model quality
4. **Visualize results** - Create diagnostic plots
5. **Save for downstream analysis** - Ready for matching, weighting, etc.

---

## 📚 Background: What are Propensity Scores?

The **propensity score** is the probability of receiving treatment given observed covariates:

```
e(x) = P(T = 1 | X)
```

Where:
- **T**: Treatment indicator (1 = received email, 0 = no email)
- **X**: Observed customer characteristics (confounders)
- **e(x)**: Propensity score (ranges from 0 to 1)

### Why Propensity Scores?

Propensity scores allow us to:
1. **Reduce dimensionality**: Replace multiple covariates with single score
2. **Enable matching**: Match treated and control units with similar scores
3. **Weight observations**: Use inverse probability weighting (IPW)
4. **Stratify**: Create strata with balanced covariates

### Key Assumptions

1. **Unconfoundedness**: T ⟂ Y(0), Y(1) | X
   - No unobserved confounders
2. **Common Support**: P(T=1 | X) ∈ (0, 1) for all X
   - Everyone has some chance of receiving treatment
3. **Correct Model**: Propensity model is correctly specified

---


## 📊 Load and Prepare Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ Libraries imported")

# Load data
data_path = '/Users/dustinober/Projects/Causal-Impact-of-Email-Marketing-on-Purchase-Behavior/data/processed/data_with_propensity_scores.csv'
data = pd.read_csv(data_path)

print(f"\n✅ Data loaded: {data.shape}")
print(f"   Columns: {list(data.columns)}")

# Quick summary
print(f"\n📊 Data Summary:")
print(f"   Total observations: {len(data):,}")
print(f"   Treatment rate: {data['received_email'].mean():.1%}")
print(f"   Purchase rate: {data['purchased_this_week_observed'].mean():.1%}")

## 🎯 Define Confounding Variables

These are the variables that affect **both** email assignment AND purchase behavior.
They create confounding bias in naive comparisons.

In [ ]:
# Define features for propensity model
features = [
    'days_since_last_purchase',     # Recency: How recently did they buy?
    'total_past_purchases',         # Frequency: How many purchases?
    'avg_order_value',              # Monetary: How much do they spend?
    'customer_tenure_weeks',        # Tenure: How long have they been a customer?
    'rfm_score'                     # Composite RFM score
]

print("📋 Confounding Variables (X):")
print("-"*70)
for i, feature in enumerate(features, 1):
    print(f"   {i}. {feature}")

print("\n💡 Why these matter:")
print("   • Recent buyers → More likely to receive emails")
print("   • Frequent buyers → More likely to receive emails")
print("   • High-value buyers → More likely to receive emails")
print("   • These differences bias naive comparisons!")

## 🔍 Step 1: Explore the Confounders

Let's examine the relationships between these variables and email receipt.

In [ ]:
# Compare means by treatment group
treated = data[data['received_email']]
control = data[~data['received_email']]

print("\n" + "="*70)
print("CONFOUNDER COMPARISON: TREATED vs CONTROL")
print("="*70)

print(f"\n{'Feature':<30} {'No Email':<15} {'Email':<15} {'Difference':<15}")
print("-"*70)

for feature in features:
    control_mean = control[feature].mean()
    treated_mean = treated[feature].mean()
    difference = treated_mean - control_mean
    
    print(f"{feature:<30} {control_mean:<15.2f} {treated_mean:<15.2f} {difference:<+15.2f}")

print("\n💡 Observation:")
print("   Email recipients have systematically different characteristics!")
print("   → This creates CONFOUNDING BIAS in naive comparisons")

## 📊 Step 2: Estimate Propensity Scores

Now we'll fit a logistic regression model to estimate P(email | X).

In [ ]:
# Prepare data for modeling
X = data[features].values
treatment = data['received_email'].values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit logistic regression
print("\n🔄 Fitting logistic regression...")
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_scaled, treatment)

# Predict propensity scores
propensity_scores = model.predict_proba(X_scaled)[:, 1]

# Add to dataframe (if not already there)
data['propensity_score'] = propensity_scores

# Model performance
auc = roc_auc_score(treatment, propensity_scores)
print(f"   ✅ Model fitted")
print(f"   📈 AUC: {auc:.3f}")

if auc > 0.7:
    performance = "Good"
elif auc > 0.6:
    performance = "Fair"
else:
    performance = "Poor"

print(f"   📊 Performance: {performance}")

## 📈 Step 3: Interpret Model Coefficients

Let's understand what drives email assignment.

In [ ]:
print("\n" + "="*70)
print("MODEL COEFFICIENTS AND ODDS RATIOS")
print("="*70)

print(f"\n{'Feature':<30} {'Coefficient':<15} {'Odds Ratio':<15} {'Effect'}")
print("-"*70)

for feature, coef in zip(features, model.coef_[0]):
    odds_ratio = np.exp(coef)
    direction = "↑" if coef > 0 else "↓"
    
    if abs(coef) > 0.3:
        strength = "Strong"
    elif abs(coef) > 0.1:
        strength = "Moderate"
    else:
        strength = "Weak"
    
    print(f"{feature:<30} {coef:<+15.4f} {odds_ratio:<15.3f} {direction} {strength}")

print("\n💡 Interpretation:")
print("   • Odds Ratio > 1: Higher feature → Higher probability of email")
print("   • Odds Ratio < 1: Higher feature → Lower probability of email")
print("   • Days since last purchase has STRONGEST effect")
print("   • Recent buyers (low days) are MUCH more likely to get emails!")

## 📏 Step 4: Propensity Score Distribution

Let's examine the distribution of propensity scores by treatment group.

In [ ]:
# Calculate summary statistics
treated_scores = data[data['received_email']]['propensity_score']
control_scores = data[~data['received_email']]['propensity_score']

print("\n" + "="*70)
print("PROPENSITY SCORE SUMMARY STATISTICS")
print("="*70)

print(f"\n{'Statistic':<20} {'Treated (Email)':<20} {'Control (No Email)':<20}")
print("-"*70)
print(f"{'Mean':<20} {treated_scores.mean():<20.4f} {control_scores.mean():<20.4f}")
print(f"{'Std':<20} {treated_scores.std():<20.4f} {control_scores.std():<20.4f}")
print(f"{'Min':<20} {treated_scores.min():<20.4f} {control_scores.min():<20.4f}")
print(f"{'25th percentile':<20} {treated_scores.quantile(0.25):<20.4f} {control_scores.quantile(0.25):<20.4f}")
print(f"{'Median':<20} {treated_scores.median():<20.4f} {control_scores.median():<20.4f}")
print(f"{'75th percentile':<20} {treated_scores.quantile(0.75):<20.4f} {control_scores.quantile(0.75):<20.4f}")
print(f"{'Max':<20} {treated_scores.max():<20.4f} {control_scores.max():<20.4f}")

print(f"\n🎯 Key Finding:")
print(f"   Mean difference: {treated_scores.mean() - control_scores.mean():.4f}")
print(f"   → Email recipients have higher propensity scores")
print(f"   → This is EXPECTED - we model P(email | X)")

## 📊 Step 5: Visualize Propensity Scores

Create comprehensive diagnostic plots.

In [ ]:
# Create diagnostic plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Plot 1: Histogram
axes[0, 0].hist(control_scores, bins=50, alpha=0.7, label='No Email',
                color='lightcoral', density=True, edgecolor='black')
axes[0, 0].hist(treated_scores, bins=50, alpha=0.7, label='Email',
                color='lightgreen', density=True, edgecolor='black')
axes[0, 0].set_xlabel('Propensity Score')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_title('Distribution by Treatment Group', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Box plot
box_data = [control_scores, treated_scores]
bp = axes[0, 1].boxplot(box_data, tick_labels=['No Email', 'Email'],
                        patch_artist=True, notch=True)
bp['boxes'][0].set_facecolor('lightcoral')
bp['boxes'][1].set_facecolor('lightgreen')
axes[0, 1].set_ylabel('Propensity Score')
axes[0, 1].set_title('Box Plots', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: ROC Curve
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(treatment, propensity_scores)
axes[0, 2].plot(fpr, tpr, color='darkgreen', linewidth=2,
                label=f'ROC (AUC = {auc:.3f})')
axes[0, 2].plot([0, 1], [0, 1], 'r--', alpha=0.7, label='Random')
axes[0, 2].set_xlabel('False Positive Rate')
axes[0, 2].set_ylabel('True Positive Rate')
axes[0, 2].set_title('ROC Curve', fontweight='bold')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# Plot 4: Feature importance
abs_coef = np.abs(model.coef_[0])
colors = ['red' if c < 0 else 'green' for c in model.coef_[0]]
bars = axes[1, 0].barh(features, model.coef_[0], color=colors, alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('Coefficient')
axes[1, 0].set_title('Feature Coefficients', fontweight='bold')
axes[1, 0].axvline(0, color='black', linestyle='-', alpha=0.5)
axes[1, 0].grid(True, alpha=0.3)

# Add value labels
for bar, coef in zip(bars, model.coef_[0]):
    axes[1, 0].text(coef + (0.005 if coef >= 0 else -0.005), bar.get_y() + bar.get_height()/2,
                    f'{coef:.3f}', ha='left' if coef >= 0 else 'right', va='center', fontsize=9)

# Plot 5: QQ plot
from scipy import stats
treated_quantiles = np.percentile(treated_scores, np.linspace(0, 100, 100))
control_quantiles = np.percentile(control_scores, np.linspace(0, 100, 100))
axes[1, 1].scatter(control_quantiles, treated_quantiles, alpha=0.6, s=20)
min_val = min(treated_quantiles.min(), control_quantiles.min())
max_val = max(treated_quantiles.max(), control_quantiles.max())
axes[1, 1].plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8)
axes[1, 1].set_xlabel('Control Quantiles')
axes[1, 1].set_ylabel('Treated Quantiles')
axes[1, 1].set_title('Q-Q Plot', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

# Plot 6: CDF
axes[1, 2].hist(control_scores, bins=50, cumulative=True, density=True,
                alpha=0.7, label='No Email', color='lightcoral',
                histtype='step', linewidth=2)
axes[1, 2].hist(treated_scores, bins=50, cumulative=True, density=True,
                alpha=0.7, label='Email', color='lightgreen',
                histtype='step', linewidth=2)
axes[1, 2].set_xlabel('Propensity Score')
axes[1, 2].set_ylabel('Cumulative Probability')
axes[1, 2].set_title('Cumulative Distribution', fontweight='bold')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Diagnostic plots created!")

## ⚖️ Step 6: Check Common Support (Overlap)

Common support means treated and control units have overlapping propensity scores.
Units without overlap cannot be matched and should be excluded.

In [ ]:
# Check overlap
min_treated = treated_scores.min()
max_treated = treated_scores.max()
min_control = control_scores.min()
max_control = control_scores.max()

overlap_min = max(min_treated, min_control)
overlap_max = min(max_treated, max_control)

print("\n" + "="*70)
print("COMMON SUPPORT CHECK")
print("="*70)

print(f"\n📏 Propensity Score Ranges:")
print(f"   Treated (email):     [{min_treated:.4f}, {max_treated:.4f}]")
print(f"   Control (no email):  [{min_control:.4f}, {max_control:.4f}]")
print(f"   Overlap region:      [{overlap_min:.4f}, {overlap_max:.4f}]")

# Units outside overlap
no_support = ((data['propensity_score'] < overlap_min) | 
              (data['propensity_score'] > overlap_max)).sum()

print(f"\n⚠️  Units without common support: {no_support:,} ({no_support/len(data)*100:.2f}%)")

if no_support > 0:
    print(f"   These units should be excluded from matching analysis")
else:
    print(f"   ✅ Perfect overlap - all units can be matched!")

# Visualize overlap
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(control_scores, bins=50, alpha=0.7, label='No Email', color='lightcoral', density=True)
plt.hist(treated_scores, bins=50, alpha=0.7, label='Email', color='lightgreen', density=True)
plt.axvline(overlap_min, color='blue', linestyle='--', linewidth=2, label='Overlap Region')
plt.axvline(overlap_max, color='blue', linestyle='--', linewidth=2)
plt.xlabel('Propensity Score')
plt.ylabel('Density')
plt.title('Propensity Score Overlap', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Show common support as shaded region
x = np.linspace(data['propensity_score'].min(), data['propensity_score'].max(), 100)
from scipy.stats import gaussian_kde
kde_control = gaussian_kde(control_scores)
kde_treated = gaussian_kde(treated_scores)

plt.fill_between(x, kde_control(x), alpha=0.5, color='lightcoral', label='No Email')
plt.fill_between(x, kde_treated(x), alpha=0.5, color='lightgreen', label='Email')
plt.axvspan(overlap_min, overlap_max, alpha=0.2, color='blue', label='Common Support')
plt.xlabel('Propensity Score')
plt.ylabel('Density')
plt.title('Common Support Region', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🎯 Step 7: Identify Extreme Scores

Extreme propensity scores (near 0 or 1) can cause problems in matching.
Let's identify units with very low or very high scores.

In [ ]:
# Identify extreme scores
low_thresholds = [1, 5]  # Bottom 1% and 5%
high_thresholds = [95, 99]  # Top 5% and 1%

print("\n" + "="*70)
print("EXTREME PROPENSITY SCORES")
print("="*70)

print(f"\n📊 Score Thresholds:")
print(f"{'Threshold':<15} {'Value':<15} {'Count':<15} {'Percentage'}")
print("-"*70)

for p in low_thresholds + high_thresholds:
    val = np.percentile(data['propensity_score'], p)
    count = (data['propensity_score'] <= val).sum() if p <= 5 else (data['propensity_score'] >= val).sum()
    pct = count / len(data) * 100
    print(f"{p}th percentile:    {val:<15.4f} {count:<15,} {pct:<15.2f}%")

# Units near boundaries
near_zero = (data['propensity_score'] < 0.01).sum()
near_one = (data['propensity_score'] > 0.99).sum()

print(f"\n🎯 Near Boundaries:")
print(f"   Score < 0.01: {near_zero:,} units ({near_zero/len(data)*100:.3f}%)")
print(f"   Score > 0.99: {near_one:,} units ({near_one/len(data)*100:.3f}%)")
print(f"   Total: {near_zero + near_one:,} units ({(near_zero + near_one)/len(data)*100:.3f}%)")

# Recommendation
if near_zero + near_one > len(data) * 0.05:
    recommendation = "Consider trimming extreme scores"
else:
    recommendation = "No trimming needed"

print(f"\n💡 Recommendation: {recommendation}")

## 💾 Step 8: Save Propensity Scores

Now we'll save the propensity scores and model for downstream analysis.

In [ ]:
# Save data with propensity scores
output_path = '/Users/dustinober/Projects/Causal-Impact-of-Email-Marketing-on-Purchase-Behavior/data/processed/data_with_propensity_scores.csv'
data.to_csv(output_path, index=False)
print(f"\n💾 Data with propensity scores saved:")
print(f"   {output_path}")
print(f"   Shape: {data.shape}")
print(f"   New column: 'propensity_score'")

# Save model parameters
model_info = {
    'features': features,
    'model_coefficients': model.coef_[0].tolist(),
    'model_intercept': model.intercept_[0].tolist(),
    'auc': float(auc),
    'scaler_mean': scaler.mean_.tolist(),
    'scaler_scale': scaler.scale_.tolist(),
    'common_support': {
        'min_treated': float(min_treated),
        'max_treated': float(max_treated),
        'min_control': float(min_control),
        'max_control': float(max_control),
        'overlap_min': float(overlap_min),
        'overlap_max': float(overlap_max)
    }
}

model_path = '/Users/dustinober/Projects/Causal-Impact-of-Email-Marketing-on-Purchase-Behavior/data/processed/propensity_model.json'
with open(model_path, 'w') as f:
    json.dump(model_info, f, indent=2)
print(f"\n💾 Model parameters saved:")
print(f"   {model_path}")

# Verify files
import os
print(f"\n✅ Files created:")
print(f"   ✓ Data file: {os.path.exists(output_path)} ({os.path.getsize(output_path)/1024/1024:.1f} MB)")
print(f"   ✓ Model file: {os.path.exists(model_path)} ({os.path.getsize(model_path):,} bytes)")

## 📝 Step 9: Summary and Interpretation

Let's summarize what we've learned.

In [ ]:
print("\n" + "="*70)
print("PROPENSITY SCORE ESTIMATION - SUMMARY")
print("="*70)

print(f"\n📊 Model Performance:")
print(f"   AUC: {auc:.3f} ({performance})")
print(f"   Sample size: {len(data):,} observations")
print(f"   Treatment rate: {data['received_email'].mean():.1%}")

print(f"\n📏 Propensity Scores:")
print(f"   Range: [{data['propensity_score'].min():.3f}, {data['propensity_score'].max():.3f}]")
print(f"   Mean (treated): {treated_scores.mean():.3f}")
print(f"   Mean (control): {control_scores.mean():.3f}")
print(f"   Difference: {treated_scores.mean() - control_scores.mean():.3f}")

print(f"\n⚖️  Common Support:")
print(f"   Overlap: [{overlap_min:.3f}, {overlap_max:.3f}]")
print(f"   Units without support: {no_support:,} ({no_support/len(data)*100:.2f}%)")

print(f"\n🎯 Key Drivers of Email Assignment:")
feature_importance = sorted(zip(features, abs(model.coef_[0])), 
                           key=lambda x: x[1], reverse=True)
for i, (feature, importance) in enumerate(feature_importance, 1):
    coef = model.coef_[0][features.index(feature)]
    direction = "↑" if coef > 0 else "↓"
    print(f"   {i}. {feature}: {direction} (|coef| = {importance:.3f})")

print(f"\n💡 Key Insights:")
print(f"   1. Days since last purchase is STRONGEST predictor")
print(f"      → Recent buyers much more likely to receive emails")
print(f"   2. Total past purchases also important")
print(f"      → Frequent buyers get more emails")
print(f"   3. Model has moderate predictive power (AUC = {auc:.3f})")
print(f"   4. Good common support - most units can be matched")
print(f"   5. Propensity scores ready for causal inference!")

print(f"\n🚀 Next Steps:")
print(f"   1. ✅ Propensity scores estimated")
print(f"   2. ✅ Common support verified")
print(f"   3. ✅ Model diagnostics complete")
print(f"   4. 🎯 Ready for Propensity Score Matching!")
print(f"   5. 🎯 Or Inverse Probability Weighting!")
print(f"   6. 🎯 Or Stratification on propensity scores!")

## 🎓 Key Takeaways

### 1. **Propensity Scores Reduce Dimensionality**
- Replace 5 covariates with 1 score
- Enable matching, weighting, stratification
- Make causal inference feasible

### 2. **Model Performance Matters**
- AUC measures predictive power
- Higher AUC = better discrimination
- Our AUC = 0.661 (moderate)

### 3. **Common Support is Critical**
- Must have overlap in propensity scores
- Units without support cannot be matched
- Check overlap before analysis

### 4. **Feature Interpretation**
- Negative coef: Higher value → Lower email probability
- Positive coef: Higher value → Higher email probability
- Odds ratios show effect magnitude

### 5. **Ready for Causal Inference**
- Propensity scores estimated ✓
- Diagnostics complete ✓
- Data saved ✓
- Can now apply matching, weighting, etc.

### 6. **Why This Matters**
- **Before**: Naive comparison shows 16.0% effect (biased!)
- **After**: Propensity scores enable causal inference
- **Goal**: Recover true 9.5% causal effect!

---

## 🚀 Next: Propensity Score Matching

Now that we have propensity scores, we can:
1. **Match** treated and control units with similar scores
2. **Calculate** treatment effect on matched sample
3. **Validate** by checking covariate balance
4. **Recover** the true causal effect!

This is the foundation for modern causal inference!

---

**Remember: Propensity scores are the bridge from correlation to causation!** ✨